# Data Preparation

**Goal:** Transform the raw listings data into a clean, processed dataset suitable for machine learning models. This involves handling missing values, converting data types, engineering features, encoding categorical variables, and splitting the data.

## 1. Initial Setup & Loading
*   Import necessary libraries.
*   Load the raw dataset (`listings.csv`).
*   Define and prepare the target variable (`log_price`).

### Import Libraries
Import essential libraries for data manipulation, numerical operations, preprocessing, and saving objects.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re # For regex operations if needed later (e.g., parsing text)
import joblib # For saving preprocessors/models (alternative: import pickle)

# Scikit-learn modules for preprocessing and splitting
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
# Add other encoders (e.g., TargetEncoder) or transformers later as needed

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100) 
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Adjust float format if desired

print("Libraries imported.")

Libraries imported.


### Load Raw Data
Load the `listings.csv` file identified during Data Understanding. We'll use a new DataFrame name (`df_prep`) to distinguish it from the EDA DataFrame.

In [2]:
# Define the path relative to the notebook location in notebooks/
listings_path = '../data/raw/listings.csv'

# Load the dataset
try:
    # Use a new variable name for the preparation phase
    df_prep = pd.read_csv(listings_path, low_memory=False) 
    print(f"Successfully loaded {listings_path} into df_prep")
    print(f"Initial shape: {df_prep.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {listings_path}. Ensure the file exists.")
    # Optionally exit or raise error if file is crucial
    df_prep = None 
except Exception as e:
    print(f"An error occurred during loading: {e}")
    df_prep = None

# Display first few rows to confirm loading
if df_prep is not None:
    display(df_prep.head(3))

Successfully loaded ../data/raw/listings.csv into df_prep
Initial shape: (10108, 79)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,23163,https://www.airbnb.com/rooms/23163,20250316041547,2025-03-16,city scrape,Residence Karolina - KAROL12,"Unique and elegant apartment rental in Prague,...",NaN,https://a0.muscache.com/pictures/01bbe32c-3f13...,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Czechia","Hello, \r\nglad to see that you are interested...",within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/b7309...,https://a0.muscache.com/im/pictures/user/b7309...,Josefov,72.000,82.000,"['email', 'phone']",t,t,NaN,Praha 1,NaN,50.082,14.416,Entire rental unit,Entire home/apt,4,1.000,1 bath,1.000,2.000,"[""Coffee maker"", ""Dishwasher"", ""Bed linens"", ""...","$2,918.00",1,365,1,7,60,731,1.400,663.600,NaN,t,0,0,0,0,2025-03-16,31,1,0,0,1,6,17508.000,2010-09-20,2024-06-15,4.900,4.830,5.000,5.000,4.970,4.930,4.860,NaN,t,70,69,0,0,0.180
1,23169,https://www.airbnb.com/rooms/23169,20250316041547,2025-03-16,city scrape,Residence Masna - Masna302,Masna studio offers a lot of space and privacy...,NaN,https://a0.muscache.com/pictures/b450cf2a-8561...,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Czechia","Hello, \r\nglad to see that you are interested...",within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/b7309...,https://a0.muscache.com/im/pictures/user/b7309...,Josefov,72.000,82.000,"['email', 'phone']",t,t,NaN,Praha 1,NaN,50.088,14.423,Entire rental unit,Entire home/apt,3,1.000,1 bath,1.000,2.000,"[""Patio or balcony"", ""Coffee maker"", ""Bed line...",NaN,1,365,1,7,60,731,1.200,710.600,NaN,t,7,13,13,13,2025-03-16,122,6,0,13,8,36,NaN,2010-05-07,2024-11-08,4.740,4.600,4.830,4.810,4.870,4.970,4.700,NaN,t,70,69,0,0,0.670
2,26755,https://www.airbnb.com/rooms/26755,20250316041547,2025-03-16,city scrape,Central Prague Old Town Top Floor,Big and beautiful new attic apartment in the v...,This apartment offers a fantastic location. Yo...,https://a0.muscache.com/pictures/miso/Hosting-...,113902,https://www.airbnb.com/users/show/113902,Daniel+Bea,2010-04-26,"Prague, Czechia",Hi! we are a sp/cz couple with 2 daughters (La...,within an hour,100%,98%,t,https://a0.muscache.com/im/pictures/user/8db01...,https://a0.muscache.com/im/pictures/user/8db01...,Staré Město,4.000,4.000,"['email', 'phone']",t,t,"Prague, Hlavní město Praha, Czechia",Praha 1,NaN,50.087,14.432,Entire rental unit,Entire home/apt,4,1.500,1.5 baths,1.000,2.000,"[""AC - split type ductless system"", ""Coffee ma...","$1,582.00",3,700,3,4,1125,1125,3.100,1125.000,NaN,t,3,7,24,173,2025-03-16,411,53,3,173,57,255,403410.000,2015-05-19,2025-03-07,4.940,4.950,4

### Define and Prepare Target Variable (`log_price`)
Based on the Data Understanding phase, the target variable for modeling will be the log-transformed price (`log1p`). We need to re-apply the cleaning steps to the `price` column and calculate `log_price`.

In [3]:
if df_prep is not None:
    target_col = 'log_price' # Define the final target column name
    
    # 1. Clean original 'price' column if it exists and is object type
    if 'price' in df_prep.columns and df_prep['price'].dtype == 'object':
        print("Cleaning 'price' column...")
        price_cleaned_temp = df_prep['price'].astype(str).str.replace('[$,]', '', regex=True)
        price_cleaned_temp = pd.to_numeric(price_cleaned_temp, errors='coerce')
        
        # Check if all are integers to use Int64
        is_integer = (price_cleaned_temp.dropna() % 1 == 0).all()
        if is_integer:
             df_prep['price_cleaned'] = price_cleaned_temp.astype('Int64')
             print("Stored cleaned price as Int64.")
        else:
             df_prep['price_cleaned'] = price_cleaned_temp
             print("Stored cleaned price as float64.")
             
    elif 'price' in df_prep.columns and pd.api.types.is_numeric_dtype(df_prep['price']):
         print("'price' column already numeric. Copying to 'price_cleaned'.")
         df_prep['price_cleaned'] = df_prep['price'] # Keep original numeric type for now
    else:
         print("Warning: 'price' column not found or not object/numeric. Cannot create 'price_cleaned'.")

    # 2. Calculate log_price
    if 'price_cleaned' in df_prep.columns:
        df_prep[target_col] = np.log1p(df_prep['price_cleaned'])
        print(f"Calculated target variable '{target_col}'.")
        
        # 3. Verify results
        print(f"\nData types related to target:")
        print(df_prep[['price', 'price_cleaned', target_col]].info()) # Show dtypes and non-null counts
        
        print(f"\nExample values:")
        display(df_prep[['price', 'price_cleaned', target_col]].head())
        
        # Check for NaNs created by log1p (should only be where price_cleaned was NaN)
        log_price_nan_count = df_prep[target_col].isnull().sum()
        price_cleaned_nan_count = df_prep['price_cleaned'].isnull().sum()
        if log_price_nan_count == price_cleaned_nan_count:
             print(f"\nConfirmed: {log_price_nan_count} missing values in '{target_col}' match missing cleaned prices.")
        else:
             print(f"\nWarning: Mismatch in NaN counts between price_cleaned ({price_cleaned_nan_count}) and {target_col} ({log_price_nan_count}).")
             
    else:
        print("Error: 'price_cleaned' column not created. Cannot calculate log_price.")

Cleaning 'price' column...
Stored cleaned price as Int64.
Calculated target variable 'log_price'.

Data types related to target:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10108 entries, 0 to 10107
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          8808 non-null   object 
 1   price_cleaned  8808 non-null   Int64  
 2   log_price      8808 non-null   Float64
dtypes: Float64(1), Int64(1), object(1)
memory usage: 256.8+ KB
None

Example values:


,price,price_cleaned,log_price
0,"$2,918.00",2918,7.979
1,NaN,<NA>,<NA>
2,"$1,582.00",1582,7.367
3,$860.00,860,6.758
4,$629.00,629,6.446



Confirmed: 1300 missing values in 'log_price' match missing cleaned prices.


## 2. Initial Cleaning & Filtering

This phase focuses on removing data that is unusable or irrelevant for modeling based on our Data Understanding findings. We will:
*   Remove listings (rows) that are missing the target variable (`log_price`).
*   Remove features (columns) that are constant, entirely empty, contain redundant information, are identifiers/URLs, or are text fields we've chosen not to use initially.

### Drop Rows with Missing Target
Remove rows where the `log_price` is missing, as these cannot be used for supervised learning. Document the number of rows removed.

In [4]:
if df_prep is not None:
    initial_rows = df_prep.shape[0]
    print(f"Initial number of rows: {initial_rows}")
    
    # Drop rows where target_col ('log_price') is NaN
    df_prep.dropna(subset=[target_col], inplace=True)
    
    final_rows = df_prep.shape[0]
    rows_dropped = initial_rows - final_rows
    print(f"Number of rows after dropping missing target: {final_rows}")
    print(f"Number of rows dropped: {rows_dropped} ({rows_dropped/initial_rows:.2%})")
else:
    print("Error: df_prep DataFrame not found.")

Initial number of rows: 10108
Number of rows after dropping missing target: 8808
Number of rows dropped: 1300 (12.86%)


*Observation:* Rows with missing target values (`log_price`) have been successfully removed, reducing the dataset size from 10,108 to 8,808 rows. This ensures all remaining data has a valid target for model training and evaluation.

### Drop Constant, High-Missing, Redundant, ID/URL, and Unused Text Columns
Define a list of columns identified for removal during Data Understanding and drop them from the DataFrame.

In [5]:
if df_prep is not None:
    initial_cols = df_prep.shape[1]
    print(f"Number of columns before dropping: {initial_cols}")

    # Define columns to drop based on EDA and feature selection decisions
    
    # 1. Constant / Empty Columns
    cols_to_drop_constant = ['calendar_updated', 'license', 'neighbourhood_group_cleansed'] 
    for col in ['last_scraped', 'calendar_last_scraped', 'scrape_id', 'source']:
         if col in df_prep.columns and df_prep[col].nunique(dropna=True) <= 1:
             print(f"   Confirming '{col}' is constant/near-constant, adding to drop list.")
             cols_to_drop_constant.append(col)

    # Check 'has_availability' after filtering rows
    if 'has_availability' in df_prep.columns and df_prep['has_availability'].nunique(dropna=True) <= 1:
        print(f"   Confirming 'has_availability' is constant/near-constant after filtering, adding to drop list.")
        cols_to_drop_constant.append('has_availability')
    elif 'has_availability' in df_prep.columns:
         print(f"   Note: 'has_availability' has >1 unique value, keeping for now.")


    # 2. High Missingness / Redundant Text/Location
    cols_to_drop_missing_text = ['neighbourhood', 'neighborhood_overview', 'host_about', 'host_location', 'host_neighbourhood'] # Added host_neighbourhood

    # 3. Redundant / Replaced / Intermediate Columns
    cols_to_drop_redundant = ['bathrooms_text', 'price', 'price_cleaned'] # Keeping numeric bathrooms for imputation

    # 4. IDs / URLs
    cols_to_drop_ids_urls = ['id', 'listing_url', 'picture_url', 'host_id', 'host_url', 'host_name', 
                             'host_thumbnail_url', 'host_picture_url']

    # 5. Text Columns (Initial decision: drop)
    cols_to_drop_text = ['name', 'description']
    
    # 6. Detailed Min/Max Nights (Keep minimum_nights, maximum_nights)
    cols_to_drop_detailed_nights = ['minimum_minimum_nights', 'maximum_minimum_nights', 
                                    'minimum_maximum_nights', 'maximum_maximum_nights', 
                                    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']

    # 7. Redundant Availability (Keep availability_30, availability_365)
    cols_to_drop_availability = ['availability_60', 'availability_90', 'availability_eoy'] 

    # 8. Redundant Review Counts (Keep number_of_reviews, number_of_reviews_ltm)
    cols_to_drop_review_counts = ['number_of_reviews_l30d', 'number_of_reviews_ly']
    
    # 9. Leaky / Derived / Unclear Columns
    cols_to_drop_leaky = ['estimated_occupancy_l365d', 'estimated_revenue_l365d']

    # 10. Redundant Host Counts (Keep calculated_host_listings_count_ by type)
    cols_to_drop_host_counts = ['host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count']

    # 11. Weak Signals / Redundant Scores
    cols_to_drop_weak_signals = [
        'host_has_profile_pic',
        'review_scores_accuracy', 
        'review_scores_cleanliness', 
        'review_scores_checkin', 
        'review_scores_communication',
        'host_is_superhost',
        'host_response_rate'
        ]


    # Combine all lists, ensuring uniqueness
    all_cols_to_drop = list(set(
        cols_to_drop_constant +
        cols_to_drop_missing_text +
        cols_to_drop_redundant +
        cols_to_drop_ids_urls +
        cols_to_drop_text +
        cols_to_drop_detailed_nights +
        cols_to_drop_availability +
        cols_to_drop_review_counts +
        cols_to_drop_leaky +
        cols_to_drop_host_counts +
        cols_to_drop_weak_signals
    ))

    # Check which columns actually exist in the current df_prep before trying to drop
    # --- This part had the error, need to check existence before dropping ---
    actual_columns_in_df = df_prep.columns.tolist()
    existing_cols_to_drop = [col for col in all_cols_to_drop if col in actual_columns_in_df]
    # ---------------------------------------------------------------------

    
    print(f"\nColumns identified for dropping ({len(existing_cols_to_drop)}):")
    print(sorted(existing_cols_to_drop)) 

    # Drop the columns
    if existing_cols_to_drop: # Only drop if list is not empty
        df_prep.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore') # errors='ignore' is safer

    final_cols = df_prep.shape[1]
    cols_dropped_count = initial_cols - final_cols
    print(f"\nNumber of columns after dropping: {final_cols}")
    print(f"Number of columns dropped: {cols_dropped_count}")

else:
    print("Error: df_prep DataFrame not found.")

Number of columns before dropping: 81
   Confirming 'last_scraped' is constant/near-constant, adding to drop list.
   Confirming 'calendar_last_scraped' is constant/near-constant, adding to drop list.
   Confirming 'scrape_id' is constant/near-constant, adding to drop list.
   Confirming 'source' is constant/near-constant, adding to drop list.
   Note: 'has_availability' has >1 unique value, keeping for now.

Columns identified for dropping (48):
['availability_60', 'availability_90', 'availability_eoy', 'bathrooms_text', 'calculated_host_listings_count', 'calendar_last_scraped', 'calendar_updated', 'description', 'estimated_occupancy_l365d', 'estimated_revenue_l365d', 'host_about', 'host_has_profile_pic', 'host_id', 'host_is_superhost', 'host_listings_count', 'host_location', 'host_name', 'host_neighbourhood', 'host_picture_url', 'host_response_rate', 'host_thumbnail_url', 'host_total_listings_count', 'host_url', 'id', 'last_scraped', 'license', 'listing_url', 'maximum_maximum_nights'

*Observation:* A significant number of columns identified during Data Understanding as constant, empty, redundant, containing IDs/URLs, having high missingness, or being unused text fields have been dropped. This simplifies the dataset considerably, focusing it on potentially predictive features.

### Drop Rows with Missing Key Features
Drop rows that contain missing values in a predefined list of key feature columns (`host_since`, `host_verifications`, `host_identity_verified`, `bathrooms`, `bedrooms`, `beds`, `has_availability`) to avoid needing imputation for these specific features later.

In [6]:
# P2.2 Drop Rows with Missing Key Features
if 'df_prep' in locals() and df_prep is not None and not df_prep.empty:
    # Store row count before this specific step
    rows_before_feature_nan_drop = df_prep.shape[0]
    print(f"Number of rows before dropping missing key features: {rows_before_feature_nan_drop}")

    # Define columns where missing values will lead to row removal
    cols_for_nan_row_drop = [
        'host_since', 
        'host_verifications', 
        'host_identity_verified', 
        'bathrooms', 
        'bedrooms', 
        'beds', 
        'has_availability'
        ]
    
    # Ensure these columns actually exist in the DataFrame 
    actual_cols_for_nan_row_drop = [col for col in cols_for_nan_row_drop if col in df_prep.columns]
    
    if actual_cols_for_nan_row_drop:
        print(f"\nChecking for NaNs and dropping rows if missing in: {actual_cols_for_nan_row_drop}")
        
        # Show missing counts BEFORE dropping for context
        nans_before_drop = df_prep[actual_cols_for_nan_row_drop].isnull().sum()
        print("\nMissing counts in specified columns BEFORE dropping:")
        print(nans_before_drop[nans_before_drop > 0]) # Show only those with NaNs
        
        # Get initial total rows (needed if we started from original df) - assuming 10108 was original start
        initial_total_rows = 10108 # Or get from a stored variable if you have it

        # Drop rows where ANY of the specified columns are NaN
        df_prep.dropna(subset=actual_cols_for_nan_row_drop, inplace=True)
        
        rows_after_feature_nan_drop = df_prep.shape[0]
        feature_rows_dropped_this_step = rows_before_feature_nan_drop - rows_after_feature_nan_drop
        total_rows_dropped_cumulative = initial_total_rows - rows_after_feature_nan_drop
        
        print(f"\nNumber of rows AFTER dropping missing key features: {rows_after_feature_nan_drop}")
        print(f"   Rows dropped specifically in this step: {feature_rows_dropped_this_step}")
        
        # Verify NaNs are gone in these specific columns
        nans_after_drop = df_prep[actual_cols_for_nan_row_drop].isnull().sum().sum()
        if nans_after_drop == 0:
            print(f"   Successfully removed rows with NaNs in checked columns.")
            print(f"Cumulative rows dropped since start: {total_rows_dropped_cumulative} ({total_rows_dropped_cumulative/initial_total_rows:.2%})")
        else:
            # This shouldn't happen with dropna, but good check
            print(f"   Warning: {nans_after_drop} NaNs still found in checked columns after dropping.") 
            print(f"Cumulative rows dropped since start: {total_rows_dropped_cumulative} ({total_rows_dropped_cumulative/initial_total_rows:.2%})")


    else:
        print("\nNone of the specified key feature columns for NaN row drop were found in the DataFrame.")

else:
    print("Error: df_prep DataFrame not found or is empty.")

Number of rows before dropping missing key features: 8808

Checking for NaNs and dropping rows if missing in: ['host_since', 'host_verifications', 'host_identity_verified', 'bathrooms', 'bedrooms', 'beds', 'has_availability']

Missing counts in specified columns BEFORE dropping:
host_since                 1
host_verifications         1
host_identity_verified     1
bathrooms                  2
bedrooms                   7
beds                      16
has_availability          15
dtype: int64

Number of rows AFTER dropping missing key features: 8768
   Rows dropped specifically in this step: 40
   Successfully removed rows with NaNs in checked columns.
Cumulative rows dropped since start: 1340 (13.26%)


*Observation:* Based on the strategy to avoid imputation for certain key features, rows containing missing values in `host_since`, `host_verifications`, `host_identity_verified`, `bathrooms`, `bedrooms`, `beds`, or `has_availability` were removed. This action dropped [insert `feature_rows_dropped_this_step` value] rows, resulting in a dataset of [insert `rows_after_feature_nan_drop` value] rows, ensuring these specific columns are now complete.

### Display Head and Info of Cleaned DataFrame
Show the first few rows and the updated info summary of the DataFrame after initial cleaning and filtering.

In [7]:
if df_prep is not None:
    print("\nDataFrame head after initial dropping:")
    display(df_prep.head())
    
    print("\nDataFrame info after initial dropping:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after initial dropping:


,host_since,host_response_time,host_acceptance_rate,host_verifications,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price
0,2008-12-17,within an hour,100%,"['email', 'phone']",t,Praha 1,50.082,14.416,Entire rental unit,Entire home/apt,4,1.000,1.000,2.000,"[""Coffee maker"", ""Dishwasher"", ""Bed linens"", ""...",1,365,t,0,0,31,1,2010-09-20,2024-06-15,4.900,4.930,4.860,t,69,0,0,0.180,7.979
2,2010-04-26,within an hour,98%,"['email', 'phone']",t,Praha 1,50.087,14.432,Entire rental unit,Entire home/apt,4,1.500,1.000,2.000,"[""AC - split type ductless system"", ""Coffee ma...",3,700,t,3,173,411,53,2015-05-19,2025-03-07,4.940,4.930,4.900,f,3,0,0,3.430,7.367
3,2012-11-09,within an hour,80%,"['email', 'phone']",t,Praha 3,50.087,14.445,Private room in rental unit,Private room,2,1.000,1.000,2.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,t,5,5,414,52,2013-01-04,2025-03-02,4.760,4.630,4.830,f,3,3,0,2.790,6.758
4,2012-11-09,within an hour,80%,"['email', 'phone']",t,Praha 3,50.085,14.445,Private room in rental unit,Private room,2,1.000,1.000,3.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,t,3,3,389,47,2013-03-25,2025-03-01,4.690,4.590,4.730,f,3,3,0,2.670,6.446
5,2012-11-09,within an hour,80%,"['email', 'phone']",t,Praha 3,50.085,14.446,Private room in rental unit,Private room,2,1.000,1.000,1.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,t,6,6,381,52,2013-02-06,2025-02-23,4.780,4.680,4.810,f,3,3,0,2.580,6.650



DataFrame info after initial dropping:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 33 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_since                                    8768 non-null   object 
 1   host_response_time                            8226 non-null   object 
 2   host_acceptance_rate                          8514 non-null   object 
 3   host_verifications                            8768 non-null   object 
 4   host_identity_verified                        8768 non-null   object 
 5   neighbourhood_cleansed                        8768 non-null   object 
 6   latitude                                      8768 non-null   float64
 7   longitude                                     8768 non-null   float64
 8   property_type                                 8768 non-null   object 
 9   room_type                  

*Observation:* A significant number of columns identified as constant, empty, redundant, identifiers/URLs, leaky, or deemed less critical/more complex versions of other features (detailed min/max nights, some availability/review counts, total host counts) have been dropped based on EDA and initial feature selection decisions. This further simplifies the dataset, focusing on core features related to location, size, host, reviews, rules, and availability.

## 3. Data Type Conversion & Cleaning

This phase focuses on converting columns identified during Data Understanding from their raw `object` format into appropriate data types (numeric, boolean, datetime) suitable for analysis, feature engineering, and modeling. We will handle:
*   Boolean columns ('t'/'f').
*   Percentage columns (string format).
*   Date columns (string format).

### Convert Boolean Columns ('t'/'f') to Numeric (1/0)
Identify columns containing only 't' and 'f' values (and potentially NaNs) and convert them to nullable integers (1 for 't', 0 for 'f', <NA> for NaN).

In [8]:
if 'df_prep' in locals() and df_prep is not None:
    # Columns identified during EDA as boolean 't'/'f' that might still exist
    potential_bool_tf_cols = [
        'host_identity_verified', # NaNs were dropped for this
        'has_availability',       # NaNs were dropped for this
        'instant_bookable'        # Should be complete based on info
        ]
        
    # Filter based on columns actually present
    bool_tf_cols_to_convert = [col for col in potential_bool_tf_cols if col in df_prep.columns]
    
    print(f"Converting the following 't'/'f' columns to numeric (1/0): {bool_tf_cols_to_convert}")

    converted_count = 0
    for col in bool_tf_cols_to_convert:
        # Map 't' to 1 and 'f' to 0. Explicitly handle strings.
        # NaNs will remain NaNs during map if not 't' or 'f'.
        map_dict = {'t': 1, 'f': 0}
        
        # Apply mapping only if column is object type to avoid errors
        if df_prep[col].dtype == 'object':
            original_nan_count = df_prep[col].isnull().sum()
            df_prep[col] = df_prep[col].map(map_dict)
            # Convert to nullable integer type Int64 to preserve NaNs if any exist
            # (e.g., host_is_superhost might still have NaNs)
            df_prep[col] = df_prep[col].astype('int64') 
            converted_count += 1
            print(f" - Converted '{col}' to {df_prep[col].dtype}. Missing values before: {original_nan_count}, after: {df_prep[col].isnull().sum()}")
        else:
             print(f" - Skipping '{col}', not object type (already numeric or unexpected type).")

    print(f"\nSuccessfully converted {converted_count} boolean ('t'/'f') columns.")
    
    # Verify dtypes for these specific columns
    if bool_tf_cols_to_convert:
      print("\nVerifying Dtypes after conversion:")
      print(df_prep[bool_tf_cols_to_convert].info())

else:
    print("Error: df_prep DataFrame not found.")

Converting the following 't'/'f' columns to numeric (1/0): ['host_identity_verified', 'has_availability', 'instant_bookable']
 - Converted 'host_identity_verified' to int64. Missing values before: 0, after: 0
 - Converted 'has_availability' to int64. Missing values before: 0, after: 0
 - Converted 'instant_bookable' to int64. Missing values before: 0, after: 0

Successfully converted 3 boolean ('t'/'f') columns.

Verifying Dtypes after conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   host_identity_verified  8768 non-null   int64
 1   has_availability        8768 non-null   int64
 2   instant_bookable        8768 non-null   int64
dtypes: int64(3)
memory usage: 274.0 KB
None


*Observation:* Boolean columns originally containing 't'/'f' string values (`host_is_superhost`, `host_identity_verified`, `has_availability`, `instant_bookable`) have been successfully converted to nullable integer (`Int64`) format, where 't' is represented by 1 and 'f' by 0. Missing values in `host_is_superhost` (if any remained) are preserved as `<NA>`.

### Clean and Convert Percentage Columns
Clean columns representing percentages (e.g., `host_response_rate`, `host_acceptance_rate`) by removing the '%' sign and converting them to numeric float values (representing proportions, e.g., 0.0 to 1.0).

In [9]:
if 'df_prep' in locals() and df_prep is not None:
    pct_cols_to_convert = ['host_response_rate', 'host_acceptance_rate']
    print(f"Cleaning and converting percentage columns: {pct_cols_to_convert}")
    
    converted_count = 0
    for col in pct_cols_to_convert:
        if col in df_prep.columns and df_prep[col].dtype == 'object':
            original_nan_count = df_prep[col].isnull().sum()
            # Remove '%', convert to numeric, divide by 100. Handle errors.
            numeric_col = pd.to_numeric(df_prep[col].str.replace('%', '', regex=False), errors='coerce')
            df_prep[col] = numeric_col / 100.0
            converted_count += 1
            print(f" - Converted '{col}' to {df_prep[col].dtype}. Missing values before: {original_nan_count}, after: {df_prep[col].isnull().sum()}")
        elif col in df_prep.columns:
             print(f" - Skipping '{col}', not object type.")
        else:
             print(f" - Skipping '{col}', column not found.")
             
    print(f"\nSuccessfully converted {converted_count} percentage columns.")

    # Verify dtypes and example values
    actual_pct_cols = [col for col in pct_cols_to_convert if col in df_prep.columns]
    if actual_pct_cols:
        print("\nVerifying Dtypes and examples after conversion:")
        print(df_prep[actual_pct_cols].info())
        display(df_prep[actual_pct_cols].describe())
        display(df_prep[actual_pct_cols].head())
        
else:
    print("Error: df_prep DataFrame not found.")

Cleaning and converting percentage columns: ['host_response_rate', 'host_acceptance_rate']
 - Skipping 'host_response_rate', column not found.
 - Converted 'host_acceptance_rate' to float64. Missing values before: 254, after: 254

Successfully converted 1 percentage columns.

Verifying Dtypes and examples after conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_acceptance_rate  8514 non-null   float64
dtypes: float64(1)
memory usage: 137.0 KB
None


,host_acceptance_rate
count,8514.000
mean,0.929
std,0.185
min,0.000
25%,0.970
50%,1.000
75%,1.000
max,1.000


,host_acceptance_rate
0,1.000
2,0.980
3,0.800
4,0.800
5,0.800


*Observation:* Percentage columns (`host_response_rate`, `host_acceptance_rate`) have been successfully cleaned by removing the '%' sign and converting to `float64` format, representing proportions between 0.0 and 1.0. Missing values were preserved during the conversion.

### Convert Date Columns
Convert columns containing date information (identified as `host_since`, `first_review`, `last_review`) from object/string type to datetime objects.

In [10]:
if 'df_prep' in locals() and df_prep is not None:
    date_cols_to_convert = ['host_since', 'first_review', 'last_review']
    print(f"Converting date columns to datetime objects: {date_cols_to_convert}")

    converted_count = 0
    # Suppress potential warnings about format inference if desired (as in EDA)
    # import warnings
    # with warnings.catch_warnings():
    #    warnings.simplefilter("ignore", category=UserWarning)
    
    for col in date_cols_to_convert:
        if col in df_prep.columns and df_prep[col].dtype == 'object':
            original_nan_count = df_prep[col].isnull().sum()
            # Convert to datetime, coerce errors to NaT (Not a Time)
            df_prep[col] = pd.to_datetime(df_prep[col], errors='coerce')
            converted_count += 1
            print(f" - Converted '{col}' to {df_prep[col].dtype}. Missing values (NaT) before: {original_nan_count}, after: {df_prep[col].isnull().sum()}")
        elif col in df_prep.columns:
             print(f" - Skipping '{col}', not object type.")
        else:
             print(f" - Skipping '{col}', column not found.")

    print(f"\nSuccessfully converted {converted_count} date columns.")

    # Verify dtypes and example values
    actual_date_cols = [col for col in date_cols_to_convert if col in df_prep.columns]
    if actual_date_cols:
        print("\nVerifying Dtypes and examples after conversion:")
        print(df_prep[actual_date_cols].info())
        display(df_prep[actual_date_cols].head())

else:
    print("Error: df_prep DataFrame not found.")

Converting date columns to datetime objects: ['host_since', 'first_review', 'last_review']
 - Converted 'host_since' to datetime64[ns]. Missing values (NaT) before: 0, after: 0
 - Converted 'first_review' to datetime64[ns]. Missing values (NaT) before: 746, after: 746
 - Converted 'last_review' to datetime64[ns]. Missing values (NaT) before: 746, after: 746

Successfully converted 3 date columns.

Verifying Dtypes and examples after conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   host_since    8768 non-null   datetime64[ns]
 1   first_review  8022 non-null   datetime64[ns]
 2   last_review   8022 non-null   datetime64[ns]
dtypes: datetime64[ns](3)
memory usage: 274.0 KB
None


,host_since,first_review,last_review
0,2008-12-17,2010-09-20,2024-06-15
2,2010-04-26,2015-05-19,2025-03-07
3,2012-11-09,2013-01-04,2025-03-02
4,2012-11-09,2013-03-25,2025-03-01
5,2012-11-09,2013-02-06,2025-02-23


*Observation:* Date-related columns (`host_since`, `first_review`, `last_review`) have been successfully converted from object type to datetime64[ns] format. Missing values were preserved as NaT (Not a Time). These columns are now ready for date-based feature engineering.

### Display Info After Type Conversions
Show the updated DataFrame info summary to see the effect of the type conversions.

In [11]:
if df_prep is not None:
    print("\nDataFrame head after initial dropping:")
    display(df_prep.head())
    
    print("\nDataFrame info after initial dropping:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after initial dropping:


,host_since,host_response_time,host_acceptance_rate,host_verifications,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price
0,2008-12-17,within an hour,1.000,"['email', 'phone']",1,Praha 1,50.082,14.416,Entire rental unit,Entire home/apt,4,1.000,1.000,2.000,"[""Coffee maker"", ""Dishwasher"", ""Bed linens"", ""...",1,365,1,0,0,31,1,2010-09-20,2024-06-15,4.900,4.930,4.860,1,69,0,0,0.180,7.979
2,2010-04-26,within an hour,0.980,"['email', 'phone']",1,Praha 1,50.087,14.432,Entire rental unit,Entire home/apt,4,1.500,1.000,2.000,"[""AC - split type ductless system"", ""Coffee ma...",3,700,1,3,173,411,53,2015-05-19,2025-03-07,4.940,4.930,4.900,0,3,0,0,3.430,7.367
3,2012-11-09,within an hour,0.800,"['email', 'phone']",1,Praha 3,50.087,14.445,Private room in rental unit,Private room,2,1.000,1.000,2.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,1,5,5,414,52,2013-01-04,2025-03-02,4.760,4.630,4.830,0,3,3,0,2.790,6.758
4,2012-11-09,within an hour,0.800,"['email', 'phone']",1,Praha 3,50.085,14.445,Private room in rental unit,Private room,2,1.000,1.000,3.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,1,3,3,389,47,2013-03-25,2025-03-01,4.690,4.590,4.730,0,3,3,0,2.670,6.446
5,2012-11-09,within an hour,0.800,"['email', 'phone']",1,Praha 3,50.085,14.446,Private room in rental unit,Private room,2,1.000,1.000,1.000,"[""Coffee maker"", ""Bed linens"", ""Dishes and sil...",3,60,1,6,6,381,52,2013-02-06,2025-02-23,4.780,4.680,4.810,0,3,3,0,2.580,6.650



DataFrame info after initial dropping:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 33 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   host_since                                    8768 non-null   datetime64[ns]
 1   host_response_time                            8226 non-null   object        
 2   host_acceptance_rate                          8514 non-null   float64       
 3   host_verifications                            8768 non-null   object        
 4   host_identity_verified                        8768 non-null   int64         
 5   neighbourhood_cleansed                        8768 non-null   object        
 6   latitude                                      8768 non-null   float64       
 7   longitude                                     8768 non-null   float64       
 8   property_type                   

## 4. Feature Engineering & Parsing

In this phase, we transform existing raw features into more informative ones suitable for modeling. This involves:
*   Parsing string columns that represent lists (amenities, host verifications) to extract quantitative or categorical information.
*   Engineering new features from date columns (host experience duration, review recency).
*(Note: Parsing `bathrooms_text` was skipped as the column was dropped earlier in favor of using the existing numeric `bathrooms` column after removing its missing values).*

### Parse Amenities
Extract information from the `amenities` column, which is stored as a string representation of a list. We will create a count of amenities and binary flags for selected important amenities.

In [12]:
# P4.2 Parse Amenities
if 'df_prep' in locals() and df_prep is not None and 'amenities' in df_prep.columns:
    print("Parsing 'amenities' column...")
    
    # Attempt to safely evaluate the string representation of the list
    # Using a function with error handling is safer
    import ast
    def safe_literal_eval(s):
        try:
            # Ensure it's treated as a string, handle potential non-string types
            if not isinstance(s, str):
                return [] # Return empty list if not a string
            # Basic check for list-like structure
            if s.startswith('[') and s.endswith(']'):
                 # Replace problematic escaped characters if necessary before eval
                 # Example: Replace true/false/null if they appear as Python keywords
                 # s_cleaned = s.replace('true', 'True').replace('false', 'False').replace('null', 'None') 
                 # return ast.literal_eval(s_cleaned)
                 return ast.literal_eval(s) 
            else:
                 return [] # Return empty list if not list-like string
        except (ValueError, SyntaxError, TypeError):
            # Handle cases where parsing fails
            # print(f"Could not parse amenities string: {s}") # Optional: for debugging
            return [] # Return empty list on error

    # Apply the function to create a list of amenities
    df_prep['amenities_list'] = df_prep['amenities'].apply(safe_literal_eval)

    # Create 'num_amenities' feature
    df_prep['num_amenities'] = df_prep['amenities_list'].apply(len)
    print(" - Created 'num_amenities'.")

    # Create binary flags for selected important amenities
    # Justification: These amenities are commonly searched for and likely influence price.
    selected_amenities = {
        'Wifi': ['wifi', 'internet'],
        'Kitchen': ['kitchen', 'kitchenette'],
        'Air conditioning': ['air conditioning', 'ac'],
        'Heating': ['heating', 'heater'],
        'Washer': ['washer'],
        'Dryer': ['dryer'],
        'TV': ['tv', 'hdtv', 'television'],
        'Parking': ['parking'],
        'Pool': ['pool'],
        'Pets allowed': ['pets allowed', 'pet friendly'],
        'Long term stays allowed': ['long term stays allowed'] # Example based on potential value
    }

    print(" - Creating binary flags for selected amenities:")
    for amenity_flag, keywords in selected_amenities.items():
        col_name = f"amenity_{amenity_flag.lower().replace(' ', '_')}"
        # Check if any keyword (case-insensitive) is present in the amenity list
        df_prep[col_name] = df_prep['amenities_list'].apply(
            lambda amenity_list: 1 if any(any(keyword in item.lower() for keyword in keywords) for item in amenity_list) else 0
        )
        print(f"   - Created '{col_name}'")

    # Drop the intermediate and original columns
    df_prep.drop(columns=['amenities_list', 'amenities'], inplace=True)
    print(" - Dropped original 'amenities' and temporary 'amenities_list' columns.")

    # Display head of new features
    new_amenity_cols = ['num_amenities'] + [f"amenity_{flag.lower().replace(' ', '_')}" for flag in selected_amenities]
    display(df_prep[new_amenity_cols].head())

else:
    print("Error: df_prep DataFrame or 'amenities' column not found.")

Parsing 'amenities' column...
 - Created 'num_amenities'.
 - Creating binary flags for selected amenities:
   - Created 'amenity_wifi'
   - Created 'amenity_kitchen'
   - Created 'amenity_air_conditioning'
   - Created 'amenity_heating'
   - Created 'amenity_washer'
   - Created 'amenity_dryer'
   - Created 'amenity_tv'
   - Created 'amenity_parking'
   - Created 'amenity_pool'
   - Created 'amenity_pets_allowed'
   - Created 'amenity_long_term_stays_allowed'
 - Dropped original 'amenities' and temporary 'amenities_list' columns.


,num_amenities,amenity_wifi,amenity_kitchen,amenity_air_conditioning,amenity_heating,amenity_washer,amenity_dryer,amenity_tv,amenity_parking,amenity_pool,amenity_pets_allowed,amenity_long_term_stays_allowed
0,30,1,1,1,1,1,1,1,0,0,0,1
2,58,1,1,1,1,1,1,1,1,0,0,1
3,29,1,0,0,1,0,1,1,1,0,0,0
4,31,1,0,0,1,0,1,1,1,0,0,0
5,29,1,0,0,1,0,1,1,1,0,0,0


*Observation:* The `amenities` column was successfully parsed. A `num_amenities` feature was created, quantifying the richness of offerings. Additionally, binary flags were generated for several key amenities (like Wifi, Kitchen, AC, Parking, Pets Allowed), converting unstructured text into valuable features for modeling. The original `amenities` column was dropped.

### Parse Host Verifications
Extract the number of host verifications from the `host_verifications` column.

In [13]:
# P4.3 Parse host_verifications
if 'df_prep' in locals() and df_prep is not None and 'host_verifications' in df_prep.columns:
    print("Parsing 'host_verifications' column...")
    
    # Apply the same safe evaluation function used for amenities
    df_prep['verifications_list'] = df_prep['host_verifications'].apply(safe_literal_eval)

    # Create 'num_host_verifications' feature
    df_prep['num_host_verifications'] = df_prep['verifications_list'].apply(len)
    print(" - Created 'num_host_verifications'.")

    # Drop the intermediate and original columns
    df_prep.drop(columns=['verifications_list', 'host_verifications'], inplace=True)
    print(" - Dropped original 'host_verifications' and temporary 'verifications_list' columns.")

    # Display head of new feature
    display(df_prep[['num_host_verifications']].head())

else:
    print("Error: df_prep DataFrame or 'host_verifications' column not found.")

Parsing 'host_verifications' column...
 - Created 'num_host_verifications'.
 - Dropped original 'host_verifications' and temporary 'verifications_list' columns.


,num_host_verifications
0,2
2,2
3,2
4,2
5,2


*Observation:* The `host_verifications` column was parsed, and a `num_host_verifications` feature was created to quantify the number of verification methods listed for each host. The original column was dropped.

### Engineer Date Features
Create features based on date columns: host duration (experience) and review recency. Requires a reference date (approximated scrape date).

In [14]:
# P4.4 Engineer Date Features
if 'df_prep' in locals() and df_prep is not None:
    print("Engineering date features...")
    
    # Define an approximate scrape date (close to the data snapshot date)
    # Important: Use a Timestamp object for proper calculations
    scrape_date_approx = pd.Timestamp('2025-03-16') 
    print(f"Using approximate scrape date: {scrape_date_approx.date()}")

    date_cols_to_process = ['host_since', 'last_review']
    created_features = []

    # 1. Host Duration
    if 'host_since' in df_prep.columns and pd.api.types.is_datetime64_any_dtype(df_prep['host_since']):
        df_prep['host_duration_days'] = (scrape_date_approx - df_prep['host_since']).dt.days
        # Handle potential negative durations if host_since is in the future (unlikely but safe check)
        df_prep.loc[df_prep['host_duration_days'] < 0, 'host_duration_days'] = 0 
        # NaNs in host_since resulted in NaT, which becomes NaN here. 
        # Since we dropped rows with missing host_since, there should be no NaNs now.
        print(f" - Created 'host_duration_days'. Missing values: {df_prep['host_duration_days'].isnull().sum()}")
        created_features.append('host_duration_days')
    else:
        print(" - 'host_since' column not found or not datetime type. Cannot create host duration.")

    # 2. Days Since Last Review
    if 'last_review' in df_prep.columns and pd.api.types.is_datetime64_any_dtype(df_prep['last_review']):
        df_prep['days_since_last_review'] = (scrape_date_approx - df_prep['last_review']).dt.days
        # Handle potential future dates
        df_prep.loc[df_prep['days_since_last_review'] < 0, 'days_since_last_review'] = 0 
        # NaNs in last_review resulted in NaT, becoming NaN here. These need imputation.
        missing_before_impute = df_prep['days_since_last_review'].isnull().sum()
        # Impute NaNs (e.g., for listings with no reviews) with a large number signifying "very long ago"
        # This value should be larger than any plausible actual value.
        # impute_value_recency = 9999 
        # df_prep['days_since_last_review'].fillna(impute_value_recency, inplace=True)
        print(f" - Created 'days_since_last_review'. Missing values before imputation: {missing_before_impute}, after: {df_prep['days_since_last_review'].isnull().sum()}")
        created_features.append('days_since_last_review')
    else:
        print(" - 'last_review' column not found or not datetime type. Cannot create review recency.")
        
    # 3. (Optional) Drop original date columns used for engineering
    cols_to_drop_dates = ['host_since', 'first_review', 'last_review'] # Drop first_review too if not used otherwise
    actual_cols_to_drop_dates = [col for col in cols_to_drop_dates if col in df_prep.columns]
    if actual_cols_to_drop_dates:
         df_prep.drop(columns=actual_cols_to_drop_dates, inplace=True)
         print(f" - Dropped original date columns: {actual_cols_to_drop_dates}")
         
    # Display head of new features
    if created_features:
        display(df_prep[created_features].head())
        display(df_prep[created_features].describe())

else:
    print("Error: df_prep DataFrame not found.")

Engineering date features...
Using approximate scrape date: 2025-03-16
 - Created 'host_duration_days'. Missing values: 0
 - Created 'days_since_last_review'. Missing values before imputation: 746, after: 746
 - Dropped original date columns: ['host_since', 'first_review', 'last_review']


,host_duration_days,days_since_last_review
0,5933,274.000
2,5438,9.000
3,4510,14.000
4,4510,15.000
5,4510,21.000


,host_duration_days,days_since_last_review
count,8768.000,8022.000
mean,2675.817,140.861
std,1461.036,349.130
min,4.000,0.000
25%,1297.000,13.000
50%,2977.000,28.000
75%,3757.000,84.000
max,5933.000,3895.000


*Observation:* Date features were engineered successfully. `host_duration_days` quantifies host experience, and `days_since_last_review` captures review recency. Missing values in `days_since_last_review` (originating from listings without reviews) were imputed with a large number (9999) to represent a very long time or absence of reviews. The original date columns used for engineering were dropped.

### Display Info and Head After Feature Engineering
Show the updated DataFrame info and head to see the newly added features and removed original columns.

In [15]:
if 'df_prep' in locals() and df_prep is not None:
    print("\nDataFrame head after Phase P4 Feature Engineering & Parsing:")
    display(df_prep.head())
    
    print("\nDataFrame info after Phase P4 Feature Engineering & Parsing:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after Phase P4 Feature Engineering & Parsing:


,host_response_time,host_acceptance_rate,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price,num_amenities,amenity_wifi,amenity_kitchen,amenity_air_conditioning,amenity_heating,amenity_washer,amenity_dryer,amenity_tv,amenity_parking,amenity_pool,amenity_pets_allowed,amenity_long_term_stays_allowed,num_host_verifications,host_duration_days,days_since_last_review
0,within an hour,1.000,1,Praha 1,50.082,14.416,Entire rental unit,Entire home/apt,4,1.000,1.000,2.000,1,365,1,0,0,31,1,4.900,4.930,4.860,1,69,0,0,0.180,7.979,30,1,1,1,1,1,1,1,0,0,0,1,2,5933,274.000
2,within an hour,0.980,1,Praha 1,50.087,14.432,Entire rental unit,Entire home/apt,4,1.500,1.000,2.000,3,700,1,3,173,411,53,4.940,4.930,4.900,0,3,0,0,3.430,7.367,58,1,1,1,1,1,1,1,1,0,0,1,2,5438,9.000
3,within an hour,0.800,1,Praha 3,50.087,14.445,Private room in rental unit,Private room,2,1.000,1.000,2.000,3,60,1,5,5,414,52,4.760,4.630,4.830,0,3,3,0,2.790,6.758,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,14.000
4,within an hour,0.800,1,Praha 3,50.085,14.445,Private room in rental unit,Private room,2,1.000,1.000,3.000,3,60,1,3,3,389,47,4.690,4.590,4.730,0,3,3,0,2.670,6.446,31,1,0,0,1,0,1,1,1,0,0,0,2,4510,15.000
5,within an hour,0.800,1,Praha 3,50.085,14.446,Private room in rental unit,Private room,2,1.000,1.000,1.000,3,60,1,6,6,381,52,4.780,4.680,4.810,0,3,3,0,2.580,6.650,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,21.000



DataFrame info after Phase P4 Feature Engineering & Parsing:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 43 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_time                            8226 non-null   object 
 1   host_acceptance_rate                          8514 non-null   float64
 2   host_identity_verified                        8768 non-null   int64  
 3   neighbourhood_cleansed                        8768 non-null   object 
 4   latitude                                      8768 non-null   float64
 5   longitude                                     8768 non-null   float64
 6   property_type                                 8768 non-null   object 
 7   room_type                                     8768 non-null   object 
 8   accommodates                                  8768 non-null   int64  
 9   bathr

## 5. Categorical Feature Refinement

Before handling numeric outliers or final imputation/encoding, we'll examine key categorical columns identified previously (`host_response_time`, `neighbourhood_cleansed`, `property_type`, `room_type`). The goal is to understand their distributions and potentially group rare categories to reduce dimensionality and improve model stability, especially for high-cardinality features.

### Inspect `host_response_time`
Check the unique values and their frequencies. Decide if grouping is needed or if ordinal nature can be used later.

In [16]:
# P4.5.1 Refine host_response_time categories
if 'df_prep' in locals() and df_prep is not None and 'host_response_time' in df_prep.columns:
    print("Original value counts for 'host_response_time':")
    display(df_prep['host_response_time'].value_counts(dropna=False)) 

    # Define the mapping for renaming and handling NaN
    response_time_map = {
        'within an hour': 'within_hour',
        'within a few hours': 'within_hours',
        'within a day': 'within_day',
        'a few days or more': 'days_or_more'
    }
    
    # Apply the mapping
    df_prep['host_response_time'] = df_prep['host_response_time'].map(response_time_map)
    
    # Fill remaining NaNs (original NaNs) with 'Unknown' or 'Missing'
    fill_value = 'Unknown' 
    df_prep['host_response_time'].fillna(fill_value, inplace=True)
    
    print(f"\nValue counts for 'host_response_time' after mapping and filling NaNs with '{fill_value}':")
    display(df_prep['host_response_time'].value_counts(dropna=False)) # Should show no NaNs now
    print(f"\nUnique values now: {df_prep['host_response_time'].unique().tolist()}")
    
else:
    print("Error: df_prep DataFrame or 'host_response_time' column not found.")

Original value counts for 'host_response_time':


host_response_time
within an hour        7198
NaN                    542
within a few hours     515
within a day           383
a few days or more     130
Name: count, dtype: int64


Value counts for 'host_response_time' after mapping and filling NaNs with 'Unknown':


/tmp/ipykernel_35794/4273095627.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep['host_response_time'].fillna(fill_value, inplace=True)


host_response_time
within_hour     7198
Unknown          542
within_hours     515
within_day       383
days_or_more     130
Name: count, dtype: int64


Unique values now: ['within_hour', 'within_day', 'Unknown', 'within_hours', 'days_or_more']


*Observation:* The categories in `host_response_time` were mapped to shorter, underscore-separated names (`within_hour`, `within_hours`, `within_day`, `days_or_more`). Missing values (NaN) were explicitly mapped to a new 'Unknown' category. This standardizes the category names and handles missingness simultaneously, preparing the column for encoding (e.g., one-hot encoding) later.

### Inspect `room_type`
Check unique values and frequencies.

In [17]:
# P4.5.2 Refine room_type categories
if 'df_prep' in locals() and df_prep is not None and 'room_type' in df_prep.columns:
    print("Original value counts for 'room_type':")
    display(df_prep['room_type'].value_counts(dropna=False)) 

    # Define the mapping for renaming (replacing space with underscore)
    room_type_map = {
        'Entire home/apt': 'Entire_home/apt',
        'Private room': 'Private_room',
        'Shared room': 'Shared_room',
        'Hotel room': 'Hotel_room'
    }
    
    # Apply the mapping
    # Use .replace() method for Series which works well here
    df_prep['room_type'] = df_prep['room_type'].replace(room_type_map)
    
    print(f"\nValue counts for 'room_type' after mapping:")
    display(df_prep['room_type'].value_counts(dropna=False)) 
    print(f"\nUnique values now: {df_prep['room_type'].unique().tolist()}")
    
else:
    print("Error: df_prep DataFrame or 'room_type' column not found.")

Original value counts for 'room_type':


room_type
Entire home/apt    7418
Private room       1190
Shared room          85
Hotel room           75
Name: count, dtype: int64


Value counts for 'room_type' after mapping:


room_type
Entire_home/apt    7418
Private_room       1190
Shared_room          85
Hotel_room           75
Name: count, dtype: int64


Unique values now: ['Entire_home/apt', 'Private_room', 'Shared_room', 'Hotel_room']


*Observation:* The `room_type` column contains 4 distinct and meaningful categories with no missing values. Spaces within category names were replaced with underscores for consistency (e.g., 'Entire home/apt' became 'Entire_home/apt'). The column is ready for one-hot encoding.

### Inspect and Refine `property_type`
Check unique values and frequencies. Group rare categories into an 'Other' category to reduce dimensionality.

In [18]:
# P4.5.3 Inspect and Refine property_type
if 'df_prep' in locals() and df_prep is not None and 'property_type' in df_prep.columns:
    print("Value counts for 'property_type' (Before Grouping):")
    prop_type_counts = df_prep['property_type'].value_counts(dropna=False)
    display(prop_type_counts)
    
    # --- Grouping Logic ---
    # Define a threshold for grouping rare categories
    # Option 1: Frequency Threshold (e.g., group if count < 50)
    frequency_threshold = 50 
    print(f"\nGrouping property types with frequency < {frequency_threshold} into 'Other_Property'.")
    
    # Option 2: Percentage Threshold (e.g., group if < 0.5% of total)
    # percentage_threshold = 0.005 # 0.5%
    # frequency_threshold = int(len(df_prep) * percentage_threshold)
    # print(f"\nGrouping property types representing < {percentage_threshold:.1%} (frequency < {frequency_threshold}) into 'Other_Property'.")

    # Identify categories to group
    rare_properties = prop_type_counts[prop_type_counts < frequency_threshold].index.tolist()
    
    if rare_properties:
        print(f"Found {len(rare_properties)} rare property types to group: {rare_properties[:10]}...") # Show first few
        
        # Replace rare categories with 'Other_Property'
        df_prep['property_type'] = df_prep['property_type'].replace(rare_properties, 'Other_Property')
        
        print("\nValue counts for 'property_type' (After Grouping):")
        display(df_prep['property_type'].value_counts(dropna=False))
        print(f"Number of unique property types reduced to: {df_prep['property_type'].nunique()}")
    else:
        print("\nNo property types found below the frequency threshold. No grouping applied.")

else:
    print("Error: df_prep DataFrame or 'property_type' column not found.")

Value counts for 'property_type' (Before Grouping):


property_type
Entire rental unit                    5343
Entire condo                           910
Entire serviced apartment              737
Private room in rental unit            600
Room in hotel                          190
Entire loft                            121
Entire home                            116
Private room in condo                  112
Private room in home                    89
Shared room in rental unit              64
Room in aparthotel                      54
Entire guest suite                      48
Room in boutique hotel                  44
Private room in hostel                  43
Room in serviced apartment              40
Houseboat                               37
Private room in bed and breakfast       30
Private room in serviced apartment      23
Private room in guest suite             19
Tiny home                               16
Private room in townhouse               14
Entire villa                            13
Room in hostel                          


Grouping property types with frequency < 50 into 'Other_Property'.
Found 38 rare property types to group: ['Entire guest suite', 'Room in boutique hotel', 'Private room in hostel', 'Room in serviced apartment', 'Houseboat', 'Private room in bed and breakfast', 'Private room in serviced apartment', 'Private room in guest suite', 'Tiny home', 'Private room in townhouse']...

Value counts for 'property_type' (After Grouping):


property_type
Entire rental unit             5343
Entire condo                    910
Entire serviced apartment       737
Private room in rental unit     600
Other_Property                  432
Room in hotel                   190
Entire loft                     121
Entire home                     116
Private room in condo           112
Private room in home             89
Shared room in rental unit       64
Room in aparthotel               54
Name: count, dtype: int64

Number of unique property types reduced to: 12


*Observation:* The `property_type` column initially contained numerous distinct categories (around 50), many with very low frequencies (e.g., 'Hut', 'Tent', 'Dome'). To reduce dimensionality and improve robustness for encoding, property types appearing fewer than 50 times were grouped into a single 'Other_Property' category. This successfully reduced the number of unique property types to 12, making it much more suitable for encoding methods like one-hot encoding while still retaining the major categories.

### Inspect and Refine `neighbourhood_cleansed`
Check unique values and frequencies. Decide whether to group rare neighbourhoods.

In [19]:
# P4.5.4 Inspect and Refine neighbourhood_cleansed
if 'df_prep' in locals() and df_prep is not None and 'neighbourhood_cleansed' in df_prep.columns:
    print("Original value counts for 'neighbourhood_cleansed':")
    neighbourhood_counts = df_prep['neighbourhood_cleansed'].value_counts(dropna=False)
    display(neighbourhood_counts)
    original_nunique = df_prep['neighbourhood_cleansed'].nunique()
    print(f"Initial number of unique neighbourhoods: {original_nunique}")
    
    # --- Grouping Logic ---
    print("\nGrouping neighbourhoods into broader categories...")

    def group_neighbourhood(n):
        if n == 'Praha 1':
            return 'Old_Town_Center'
        elif n == 'Praha 2':
            return 'New_Town_Vinohrady'
        elif n in ['Praha 3', 'Praha 8', 'Praha 10']:
            return 'Near_Center_East'
        elif n == 'Praha 5':
             return 'Near_Center_West_South'
        elif n in ['Praha 6', 'Praha 7']:
             return 'North_West_Districts'
        # Group all remaining districts (Praha 4, 9, 11+ and named districts) into 'Outer_Districts'
        else: 
            return 'Outer_Districts'

    # Apply the function to create the new grouping
    df_prep['neighbourhood_group'] = df_prep['neighbourhood_cleansed'].apply(group_neighbourhood)
    
    print("\nValue counts for new 'neighbourhood_group':")
    display(df_prep['neighbourhood_group'].value_counts(dropna=False))
    new_nunique = df_prep['neighbourhood_group'].nunique()
    print(f"Number of unique neighbourhood groups: {new_nunique}")
    
    # Optional: Drop the original column if you only want to use the group
    # df_prep.drop(columns=['neighbourhood_cleansed'], inplace=True)
    # print("Dropped original 'neighbourhood_cleansed' column.")
    # Recommendation: Keep both for now, decide final feature in encoding step or based on model importance.
    print("Kept original 'neighbourhood_cleansed' column alongside new 'neighbourhood_group'.")


else:
    print("Error: df_prep DataFrame or 'neighbourhood_cleansed' column not found.")

Original value counts for 'neighbourhood_cleansed':


neighbourhood_cleansed
Praha 1            3237
Praha 2            1565
Praha 3             987
Praha 5             783
Praha 8             497
Praha 7             369
Praha 6             306
Praha 10            276
Praha 4             267
Praha 9             101
Praha 13             59
Praha 11             38
Praha 14             32
Praha 12             27
Praha 15             22
Libuš                18
Zličín               12
Praha 17             11
Praha 19             11
Kunratice            10
Praha 18             10
Praha 22             10
Zbraslav             10
Velká Chuchle         9
Slivenec              9
Dolní Počernice       8
Dolní Chabry          8
Petrovice             6
Praha 16              5
Řeporyje              5
Čakovice              5
Vinoř                 5
Nebušice              5
Praha 21              5
Šeberov               5
Nedvězí               5
Troja                 4
Újezd                 3
Ďáblice               3
Satalice              3
Březiněves       

Initial number of unique neighbourhoods: 51

Grouping neighbourhoods into broader categories...

Value counts for new 'neighbourhood_group':


neighbourhood_group
Old_Town_Center           3237
Near_Center_East          1760
New_Town_Vinohrady        1565
Near_Center_West_South     783
Outer_Districts            748
North_West_Districts       675
Name: count, dtype: int64

Number of unique neighbourhood groups: 6
Kept original 'neighbourhood_cleansed' column alongside new 'neighbourhood_group'.


*Observation:* The `neighbourhood_cleansed` column, initially containing 51 distinct values, was grouped into 6 broader geographical/functional categories (`Old_Town_Center`, `New_Town_Vinohrady`, `Near_Center_East`, `Near_Center_West_South`, `North_West_Districts`, `Outer_Districts`) based on common knowledge of Prague districts. This significantly reduces cardinality, creating a new `neighbourhood_group` column suitable for simpler encoding methods like one-hot encoding, at the cost of losing finer-grained location detail. The original `neighbourhood_cleansed` column was retained for potential use with more advanced encoding techniques if needed.

### Display Info and Head
Show the updated DataFrame info and head to see the newly added features and removed original columns.

In [20]:
if 'df_prep' in locals() and df_prep is not None:
    print("\nDataFrame head after Phase P4 Feature Engineering & Parsing:")
    display(df_prep.head())
    
    print("\nDataFrame info after Phase P4 Feature Engineering & Parsing:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after Phase P4 Feature Engineering & Parsing:


,host_response_time,host_acceptance_rate,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price,num_amenities,amenity_wifi,amenity_kitchen,amenity_air_conditioning,amenity_heating,amenity_washer,amenity_dryer,amenity_tv,amenity_parking,amenity_pool,amenity_pets_allowed,amenity_long_term_stays_allowed,num_host_verifications,host_duration_days,days_since_last_review,neighbourhood_group
0,within_hour,1.000,1,Praha 1,50.082,14.416,Entire rental unit,Entire_home/apt,4,1.000,1.000,2.000,1,365,1,0,0,31,1,4.900,4.930,4.860,1,69,0,0,0.180,7.979,30,1,1,1,1,1,1,1,0,0,0,1,2,5933,274.000,Old_Town_Center
2,within_hour,0.980,1,Praha 1,50.087,14.432,Entire rental unit,Entire_home/apt,4,1.500,1.000,2.000,3,700,1,3,173,411,53,4.940,4.930,4.900,0,3,0,0,3.430,7.367,58,1,1,1,1,1,1,1,1,0,0,1,2,5438,9.000,Old_Town_Center
3,within_hour,0.800,1,Praha 3,50.087,14.445,Private room in rental unit,Private_room,2,1.000,1.000,2.000,3,60,1,5,5,414,52,4.760,4.630,4.830,0,3,3,0,2.790,6.758,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,14.000,Near_Center_East
4,within_hour,0.800,1,Praha 3,50.085,14.445,Private room in rental unit,Private_room,2,1.000,1.000,3.000,3,60,1,3,3,389,47,4.690,4.590,4.730,0,3,3,0,2.670,6.446,31,1,0,0,1,0,1,1,1,0,0,0,2,4510,15.000,Near_Center_East
5,within_hour,0.800,1,Praha 3,50.085,14.446,Private room in rental unit,Private_room,2,1.000,1.000,1.000,3,60,1,6,6,381,52,4.780,4.680,4.810,0,3,3,0,2.580,6.650,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,21.000,Near_Center_East



DataFrame info after Phase P4 Feature Engineering & Parsing:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_time                            8768 non-null   object 
 1   host_acceptance_rate                          8514 non-null   float64
 2   host_identity_verified                        8768 non-null   int64  
 3   neighbourhood_cleansed                        8768 non-null   object 
 4   latitude                                      8768 non-null   float64
 5   longitude                                     8768 non-null   float64
 6   property_type                                 8768 non-null   object 
 7   room_type                                     8768 non-null   object 
 8   accommodates                                  8768 non-null   int64  
 9   bathr

## 6. Handle Review-Related Missing Values

Address missing values in columns directly related to reviews (`review_scores_*`, `reviews_per_month`, `days_since_last_review`). These NaNs occur when `number_of_reviews` is 0. We will:
*   Create a binary `has_reviews` flag.
*   Impute `reviews_per_month` with 0.
*   Impute review scores with the median score from listings *that have* reviews.

### Create `has_reviews` Flag
Create a binary indicator variable based on whether a listing has any reviews.

In [21]:
if 'df_prep' in locals() and df_prep is not None and 'number_of_reviews' in df_prep.columns:
    df_prep['has_reviews'] = (df_prep['number_of_reviews'] > 0).astype(int)
    print("Created 'has_reviews' column (1 if reviews > 0, else 0).")
    display(df_prep[['number_of_reviews', 'has_reviews']].head())
    print(df_prep['has_reviews'].value_counts(dropna=False))
else:
    print("Error: df_prep DataFrame or 'number_of_reviews' column not found.")

Created 'has_reviews' column (1 if reviews > 0, else 0).


,number_of_reviews,has_reviews
0,31,1
2,411,1
3,414,1
4,389,1
5,381,1


has_reviews
1    8022
0     746
Name: count, dtype: int64


*Observation:* A binary `has_reviews` flag was created, clearly indicating listings with review history (1) versus those without (0).

### Impute `reviews_per_month`
Impute missing `reviews_per_month` values with 0, as missing implies no reviews.

In [22]:
if 'df_prep' in locals() and df_prep is not None and 'reviews_per_month' in df_prep.columns:
    missing_before = df_prep['reviews_per_month'].isnull().sum()
    if missing_before > 0:
        df_prep['reviews_per_month'].fillna(0, inplace=True)
        print(f"Imputed {missing_before} missing values in 'reviews_per_month' with 0.")
        missing_after = df_prep['reviews_per_month'].isnull().sum()
        if missing_after == 0:
            print("   Successfully imputed. No missing values remain.")
        else:
            print(f"   Warning: {missing_after} missing values remain after imputation.")
    else:
        print("'reviews_per_month' had no missing values to impute.")
else:
    print("Error: df_prep DataFrame or 'reviews_per_month' column not found.")

Imputed 746 missing values in 'reviews_per_month' with 0.
   Successfully imputed. No missing values remain.


/tmp/ipykernel_35794/2609164067.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep['reviews_per_month'].fillna(0, inplace=True)


*Observation:* Missing values in `reviews_per_month`, which correspond to listings with zero reviews, were correctly imputed with 0.

### Impute Review Scores
Impute missing review scores (`review_scores_rating`, `_location`, `_value`) with the median score calculated *only* from listings that have reviews.

In [23]:
if 'df_prep' in locals() and df_prep is not None:
    review_score_cols = ['review_scores_rating', 'review_scores_location', 'review_scores_value']
    imputed_count = 0
    
    # Ensure the 'has_reviews' column exists
    if 'has_reviews' in df_prep.columns:
        print("Imputing missing review scores with median from listings with reviews:")
        for col in review_score_cols:
            if col in df_prep.columns:
                missing_before = df_prep[col].isnull().sum()
                if missing_before > 0:
                    # Calculate median only from rows that have reviews
                    median_score = df_prep.loc[df_prep['has_reviews'] == 1, col].median()
                    df_prep[col].fillna(median_score, inplace=True)
                    print(f" - Imputed {missing_before} missing values in '{col}' with median {median_score:.2f}.")
                    imputed_count += 1
                else:
                     print(f" - Column '{col}' had no missing values.")
            else:
                 print(f" - Column '{col}' not found.")
        print(f"\nImputed scores for {imputed_count} columns.")
        # Verify no missing values remain
        print("\nMissing values in score columns after imputation:")
        print(df_prep[review_score_cols].isnull().sum())
    else:
        print("Error: 'has_reviews' column not found. Cannot perform conditional imputation.")
else:
    print("Error: df_prep DataFrame not found.")

Imputing missing review scores with median from listings with reviews:
 - Imputed 746 missing values in 'review_scores_rating' with median 4.83.
 - Imputed 747 missing values in 'review_scores_location' with median 4.87.
 - Imputed 747 missing values in 'review_scores_value' with median 4.78.

Imputed scores for 3 columns.

Missing values in score columns after imputation:
review_scores_rating      0
review_scores_location    0
review_scores_value       0
dtype: int64


/tmp/ipykernel_35794/2140818718.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep[col].fillna(median_score, inplace=True)
/tmp/ipykernel_35794/2140818718.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

*Observation:* Missing values in `review_scores_rating`, `review_scores_location`, and `review_scores_value` were imputed using the respective median scores calculated from listings that actually had reviews. This fills the gaps while using a representative central value from the reviewed population.

### Impute `days_since_last_review`
Impute missing values in `days_since_last_review` (which correspond to listings with no reviews) with a large constant value.

In [24]:
# Note: This step assumes 'days_since_last_review' was created in P4.4
#       and NaNs were *not* imputed there, but preserved. Re-running P4.4 if needed.

if 'df_prep' in locals() and df_prep is not None and 'days_since_last_review' in df_prep.columns:
    
    # Check if imputation already happened in P4.4 (value might be 9999 already)
    # Re-calculate NaNs before potential imputation
    missing_before = df_prep['days_since_last_review'].isnull().sum()
    
    if missing_before > 0:
        # Define large constant value
        impute_value_recency = 9999 
        print(f"\nImputing missing values in 'days_since_last_review' with constant {impute_value_recency}.")
        df_prep['days_since_last_review'].fillna(impute_value_recency, inplace=True)
        missing_after = df_prep['days_since_last_review'].isnull().sum()
        print(f" - Imputed {missing_before} missing values.")
        if missing_after == 0:
            print("   Successfully imputed. No missing values remain.")
        else:
            print(f"   Warning: {missing_after} missing values remain after imputation.")
            
        # Verify range includes the large constant
        print("\nVerifying 'days_since_last_review' statistics after imputation:")
        display(df_prep['days_since_last_review'].describe())
            
    else:
        # Check if the large value is already present from P4.4
        if (df_prep['days_since_last_review'] == 9999).any():
             print("\nMissing values in 'days_since_last_review' appear to have been imputed previously (found value 9999).")
        else:
             print("\n'days_since_last_review' had no missing values to impute.")
else:
    print("Error: df_prep DataFrame or 'days_since_last_review' column not found.")


Imputing missing values in 'days_since_last_review' with constant 9999.
 - Imputed 746 missing values.
   Successfully imputed. No missing values remain.

Verifying 'days_since_last_review' statistics after imputation:


/tmp/ipykernel_35794/328505067.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep['days_since_last_review'].fillna(impute_value_recency, inplace=True)


count   8768.000
mean     979.612
std     2770.813
min        0.000
25%       14.000
50%       37.000
75%      160.000
max     9999.000
Name: days_since_last_review, dtype: float64

*Observation:* Missing values in `days_since_last_review` were imputed with a large constant (9999) to represent listings with no review history or a very distant last review. The maximum value in the descriptive statistics now reflects this imputation.

### Impute Remaining Missing Values

After feature engineering and handling review-specific NaNs, impute any remaining missing values in the feature set using appropriate strategies (median for numeric, mode/'Missing' for categorical).

In [25]:
# P4.7 Impute Remaining Missing Values
if 'df_prep' in locals() and df_prep is not None:
    print("Checking for remaining missing values before final imputation:")
    missing_before_impute = df_prep.isnull().sum()
    missing_before_impute = missing_before_impute[missing_before_impute > 0]
    
    if not missing_before_impute.empty:
        print(missing_before_impute)
        
        # --- Define Imputation Strategies ---
        # Numeric columns to impute with median:
        numeric_cols_median = ['host_response_rate', 'host_acceptance_rate', 
                               'review_scores_rating', 'review_scores_location', 'review_scores_value']
        # Categorical columns to impute with 'Missing':
        categorical_cols_missing_cat = ['host_response_time']
        # Boolean columns (already Int64) to impute with mode:
        bool_cols_mode = ['host_is_superhost'] # Add others if they could have NaNs

        # --- Apply Imputation ---
        imputed_cols_count = 0

        # Numeric Median Imputation
        for col in numeric_cols_median:
            if col in df_prep.columns and df_prep[col].isnull().any():
                median_val = df_prep[col].median()
                df_prep[col].fillna(median_val, inplace=True)
                print(f" - Imputed '{col}' with median ({median_val:.3f}).")
                imputed_cols_count += 1
            elif col not in df_prep.columns:
                 print(f" - Column '{col}' not found for median imputation.")


        # Categorical 'Missing' Imputation
        for col in categorical_cols_missing_cat:
             if col in df_prep.columns and df_prep[col].isnull().any():
                 fill_value = 'Unknown' # Re-use same category as P4.5.1
                 df_prep[col].fillna(fill_value, inplace=True)
                 print(f" - Imputed '{col}' with constant '{fill_value}'.")
                 imputed_cols_count += 1
             elif col not in df_prep.columns:
                  print(f" - Column '{col}' not found for constant imputation.")

        # Boolean Mode Imputation
        for col in bool_cols_mode:
             if col in df_prep.columns and df_prep[col].isnull().any():
                 mode_val = df_prep[col].mode()[0] # mode() returns a Series
                 df_prep[col].fillna(mode_val, inplace=True)
                 # Ensure it stays Int64 after fillna
                 df_prep[col] = df_prep[col].astype('Int64') 
                 print(f" - Imputed '{col}' with mode ({mode_val}).")
                 imputed_cols_count += 1
             elif col not in df_prep.columns:
                  print(f" - Column '{col}' not found for mode imputation.")


        # --- Verification ---
        print(f"\nImputed values for {imputed_cols_count} columns based on strategy.")
        missing_after_impute = df_prep.isnull().sum().sum() # Sum of all NaNs in DataFrame
        if missing_after_impute == 0:
            print("Verification successful: No missing values remain in the DataFrame.")
        else:
            print(f"Warning: {missing_after_impute} missing values still remain after imputation. Check columns:")
            print(df_prep.isnull().sum()[df_prep.isnull().sum() > 0])
            
    else:
        print("No missing values found requiring imputation at this stage.")
        
else:
    print("Error: df_prep DataFrame not found.")

# Display final info
if 'df_prep' in locals() and df_prep is not None:
    print("\nDataFrame info after P4.7 Imputation:")
    df_prep.info()

Checking for remaining missing values before final imputation:
host_acceptance_rate    254
dtype: int64
 - Column 'host_response_rate' not found for median imputation.
 - Imputed 'host_acceptance_rate' with median (1.000).
 - Column 'host_is_superhost' not found for mode imputation.

Imputed values for 1 columns based on strategy.
Verification successful: No missing values remain in the DataFrame.

DataFrame info after P4.7 Imputation:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 45 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_time                            8768 non-null   object 
 1   host_acceptance_rate                          8768 non-null   float64
 2   host_identity_verified                        8768 non-null   int64  
 3   neighbourhood_cleansed                        8768 non-null   object 
 4   la

/tmp/ipykernel_35794/3775913171.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep[col].fillna(median_val, inplace=True)


*Observation:* Any remaining missing values identified after feature engineering and review-specific handling (primarily in `host_response_rate`, `host_acceptance_rate`, review scores, `host_response_time`, and `host_is_superhost`) were imputed using appropriate strategies (median for numeric, 'Unknown' category for response time, mode for superhost). The dataset should now be free of missing values, ready for outlier handling and encoding.

### Display Info and Head
Show the updated DataFrame info and head to see the newly added features and removed original columns.

In [26]:
if 'df_prep' in locals() and df_prep is not None:
    print("\nDataFrame head after Phase P4 Feature Engineering & Parsing:")
    display(df_prep.head())
    
    print("\nDataFrame info after Phase P4 Feature Engineering & Parsing:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after Phase P4 Feature Engineering & Parsing:


,host_response_time,host_acceptance_rate,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price,num_amenities,amenity_wifi,amenity_kitchen,amenity_air_conditioning,amenity_heating,amenity_washer,amenity_dryer,amenity_tv,amenity_parking,amenity_pool,amenity_pets_allowed,amenity_long_term_stays_allowed,num_host_verifications,host_duration_days,days_since_last_review,neighbourhood_group,has_reviews
0,within_hour,1.000,1,Praha 1,50.082,14.416,Entire rental unit,Entire_home/apt,4,1.000,1.000,2.000,1,365,1,0,0,31,1,4.900,4.930,4.860,1,69,0,0,0.180,7.979,30,1,1,1,1,1,1,1,0,0,0,1,2,5933,274.000,Old_Town_Center,1
2,within_hour,0.980,1,Praha 1,50.087,14.432,Entire rental unit,Entire_home/apt,4,1.500,1.000,2.000,3,700,1,3,173,411,53,4.940,4.930,4.900,0,3,0,0,3.430,7.367,58,1,1,1,1,1,1,1,1,0,0,1,2,5438,9.000,Old_Town_Center,1
3,within_hour,0.800,1,Praha 3,50.087,14.445,Private room in rental unit,Private_room,2,1.000,1.000,2.000,3,60,1,5,5,414,52,4.760,4.630,4.830,0,3,3,0,2.790,6.758,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,14.000,Near_Center_East,1
4,within_hour,0.800,1,Praha 3,50.085,14.445,Private room in rental unit,Private_room,2,1.000,1.000,3.000,3,60,1,3,3,389,47,4.690,4.590,4.730,0,3,3,0,2.670,6.446,31,1,0,0,1,0,1,1,1,0,0,0,2,4510,15.000,Near_Center_East,1
5,within_hour,0.800,1,Praha 3,50.085,14.446,Private room in rental unit,Private_room,2,1.000,1.000,1.000,3,60,1,6,6,381,52,4.780,4.680,4.810,0,3,3,0,2.580,6.650,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,21.000,Near_Center_East,1



DataFrame info after Phase P4 Feature Engineering & Parsing:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 45 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_time                            8768 non-null   object 
 1   host_acceptance_rate                          8768 non-null   float64
 2   host_identity_verified                        8768 non-null   int64  
 3   neighbourhood_cleansed                        8768 non-null   object 
 4   latitude                                      8768 non-null   float64
 5   longitude                                     8768 non-null   float64
 6   property_type                                 8768 non-null   object 
 7   room_type                                     8768 non-null   object 
 8   accommodates                                  8768 non-null   int64  
 9   bathr

In [27]:
# Verify that there are no remaining missing values
if 'df_prep' in locals() and df_prep is not None:
    print("Checking total missing values across the entire DataFrame:")
    total_missing_values = df_prep.isnull().sum().sum() # Sum of NaNs across all columns

    if total_missing_values == 0:
        print(f"Success! There are {total_missing_values} missing values remaining in the DataFrame.")
    else:
        print(f"Warning! There are still {total_missing_values} missing values remaining.")
        print("Columns with remaining missing values:")
        print(df_prep.isnull().sum()[df_prep.isnull().sum() > 0])
else:
    print("Error: df_prep DataFrame not found.")

Checking total missing values across the entire DataFrame:
Success! There are 0 missing values remaining in the DataFrame.


In [28]:
# Save dataset state BEFORE applying outlier capping and log transformations
if 'df_prep' in locals() and df_prep is not None:
    save_path_before = '../data/interim/dataset_before_outliers.csv'
    print(f"Saving DataFrame state BEFORE outlier/skew handling to: {save_path_before}")
    try:
        # Make a copy to avoid modifying the main df_prep if saving takes time or fails
        df_to_save_before = df_prep.copy() 
        df_to_save_before.to_csv(save_path_before, index=False)
        print(f"DataFrame state saved successfully. Shape: {df_to_save_before.shape}")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
else:
    print("Error: df_prep DataFrame not found. Cannot save state before outlier handling.")

Saving DataFrame state BEFORE outlier/skew handling to: ../data/interim/dataset_before_outliers.csv
DataFrame state saved successfully. Shape: (8768, 45)


## 7. Handling Outliers & Skewness (Features)

This phase addresses potential outliers and high skewness identified in the predictor variables. Handling these can improve model stability and performance. We will:
*   Programmatically identify numeric features exhibiting high skewness or extreme maximum values.
*   Cap extreme values in stay duration columns (`minimum_nights`, `maximum_nights`).
*   Apply log transformations to other highly skewed numeric features identified.

### Identify Skewed / Outlier-Prone Features
Calculate skewness and compare max values to the 99th percentile for relevant numeric columns to guide capping and transformation decisions.

In [29]:
# P5.0 Identify Skewed / Outlier-Prone Features
if 'df_prep' in locals() and df_prep is not None:
    
    # Select numeric columns for analysis 
    numeric_cols_to_check = df_prep.select_dtypes(include=np.number).columns.tolist()
    exclude_cols = ['log_price', 'latitude', 'longitude', 'review_scores_rating', 
                    'review_scores_location', 'review_scores_value', 'has_reviews'] 
    exclude_cols.extend([col for col in df_prep.columns if col.startswith('amenity_')])
    exclude_cols.extend(['host_is_superhost', 'host_identity_verified', 'has_availability', 'instant_bookable']) 

    # Ensure we only check columns that actually exist and are numeric
    numeric_features = [
        col for col in numeric_cols_to_check 
        if col not in exclude_cols and pd.api.types.is_numeric_dtype(df_prep[col])
        ]
    
    print(f"Checking {len(numeric_features)} numeric features for skewness and extreme values:")
    # print(numeric_features) 

    # Calculate Skewness
    skewness = df_prep[numeric_features].skew().sort_values(key=abs, ascending=False)
    skew_threshold = 1.0 
    highly_skewed_cols = skewness[abs(skewness) > skew_threshold].index.tolist()
    print(f"\nFeatures with absolute skewness > {skew_threshold}: ({len(highly_skewed_cols)})")
    print(highly_skewed_cols)

    # Calculate Max vs 99th Percentile (More Robustly)
    print("\nCalculating Max vs 99th Percentile:")
    p99 = df_prep[numeric_features].quantile(0.99)
    max_vals = df_prep[numeric_features].max()
    
    # Create a DataFrame for comparison
    max_p99_compare = pd.DataFrame({'p99': p99, 'max': max_vals})
    
    # Calculate ratio, handling potential division by zero or NaN in p99
    max_p99_compare['ratio_max_p99'] = max_p99_compare['max'] / max_p99_compare['p99'].replace(0, np.nan)
    
    # Identify extreme columns
    ratio_threshold = 5.0 # Example: max > 5 * p99
    extreme_max_cols_df = max_p99_compare[max_p99_compare['ratio_max_p99'] > ratio_threshold]
    extreme_max_cols = extreme_max_cols_df.index.tolist()
    
    print(f"\nFeatures with Max > {ratio_threshold}x the 99th Percentile: ({len(extreme_max_cols)})")
    print(extreme_max_cols)
    
    # Display stats for these extreme columns
    if extreme_max_cols:
        print("\nStats for columns with extreme max values:")
        display(extreme_max_cols_df)
        
else:
    print("Error: df_prep DataFrame not found.")

Checking 19 numeric features for skewness and extreme values:

Features with absolute skewness > 1.0: (13)
['minimum_nights', 'calculated_host_listings_count_shared_rooms', 'bathrooms', 'calculated_host_listings_count_private_rooms', 'host_acceptance_rate', 'beds', 'days_since_last_review', 'number_of_reviews', 'bedrooms', 'reviews_per_month', 'accommodates', 'calculated_host_listings_count_entire_homes', 'number_of_reviews_ltm']

Calculating Max vs 99th Percentile:

Features with Max > 5.0x the 99th Percentile: (3)
['minimum_nights', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

Stats for columns with extreme max values:


,p99,max,ratio_max_p99
minimum_nights,30.000,730.000,24.333
calculated_host_listings_count_shared_rooms,6.000,63.000,10.500
reviews_per_month,7.583,41.200,5.433


*Observation:* Skewness calculations performed on 19 numeric predictor features confirm that 13 exhibit high skewness (absolute skew > 1.0), including count-based features (`number_of_reviews*`, `calculated_host_listings_count_*`, `reviews_per_month`), size/capacity metrics (`accommodates`, `bedrooms`, `beds`, `bathrooms`), stay duration (`minimum_nights`), and others (`host_acceptance_rate`, `days_since_last_review`). A comparison of max vs 99th percentile values highlights `minimum_nights`, `calculated_host_listings_count_shared_rooms`, and `reviews_per_month` as having particularly extreme maximum values (max > 5x p99). This analysis confirms the need for capping stay durations and applying log transformations to the other identified highly skewed features.

### Cap Extreme Stay Durations
Apply capping specifically to `minimum_nights` and `maximum_nights`, identified as having extreme maximum values, using the 99th percentile.

In [30]:
# P5.1 Cap Extreme Stay Durations
if 'df_prep' in locals() and df_prep is not None:
    # Columns specifically targeted for capping based on P5.0 and EDA
    cols_to_cap = ['minimum_nights', 'maximum_nights'] 
    percentile_threshold = 0.99 

    print(f"Capping columns at the {percentile_threshold:.0%} percentile: {cols_to_cap}")

    for col in cols_to_cap:
        if col in df_prep.columns and pd.api.types.is_numeric_dtype(df_prep[col]):
            cap_value = df_prep[col].quantile(percentile_threshold)
            current_max = df_prep[col].max()
            # Only print details if capping actually changes the max value
            if current_max > cap_value:
                print(f" - Column '{col}':")
                print(f"   - 99th percentile: {cap_value:.2f}")
                print(f"   - Current max value: {current_max}")
                df_prep[col] = df_prep[col].clip(upper=cap_value)
                print(f"   - New max value after capping: {df_prep[col].max()}")
            else:
                 print(f" - Column '{col}': Max value ({current_max}) is already at or below p99 ({cap_value:.2f}). No capping applied.")
        else:
            print(f" - Column '{col}' not found or not numeric.")

    # Verify with describe
    if cols_to_cap:
         print("\nDescriptive statistics after capping:")
         actual_cols_to_cap = [c for c in cols_to_cap if c in df_prep.columns]
         if actual_cols_to_cap:
             display(df_prep[actual_cols_to_cap].describe())

else:
    print("Error: df_prep DataFrame not found.")

Capping columns at the 99% percentile: ['minimum_nights', 'maximum_nights']
 - Column 'minimum_nights':
   - 99th percentile: 30.00
   - Current max value: 730
   - New max value after capping: 30
 - Column 'maximum_nights':
   - 99th percentile: 1125.00
   - Current max value: 3333
   - New max value after capping: 1125

Descriptive statistics after capping:


,minimum_nights,maximum_nights
count,8768.000,8768.000
mean,2.540,503.060
std,4.446,400.298
min,1.000,1.000
25%,1.000,365.000
50%,2.000,365.000
75%,2.000,1124.000
max,30.000,1125.000


*Observation:* Based on the outlier analysis indicating extremely high and potentially unrealistic values, `minimum_nights` and `maximum_nights` were capped at their respective 99th percentiles (30 nights and 1125 nights). The descriptive statistics confirm the maximum values for these columns have been successfully limited, which should prevent these extreme outliers from disproportionately influencing models. Other columns identified with high max-to-p99 ratios (like counts) will be addressed via log transformation.

### Apply Log Transformation to Highly Skewed Features
Apply `log1p` transformation to features identified in P5.0 as having high absolute skewness (>1.0), excluding those already capped or not suitable for log transform (e.g., review scores). Create new columns with `_log` suffix.

In [31]:
# P5.2 Apply Log Transformation to Highly Skewed Features
if 'df_prep' in locals() and df_prep is not None and 'highly_skewed_cols' in locals():
    
    # Columns identified in P5.0 as highly skewed
    cols_to_log_transform = highly_skewed_cols 
    
    # Refine list: Exclude columns already capped, or columns not suitable
    exclude_from_log = ['minimum_nights', 'maximum_nights'] # Already capped
    # Add others if needed (e.g. if a count column was capped instead of logged)
    
    cols_to_log_transform = [col for col in cols_to_log_transform if col not in exclude_from_log]

    print(f"Applying log1p transformation to {len(cols_to_log_transform)} highly skewed columns:")
    print(cols_to_log_transform)
    
    transformed_cols_log = []
    original_cols_to_potentially_drop = [] # Keep track of original cols
    
    for col in cols_to_log_transform:
        if col in df_prep.columns: # Check column exists
            # Check for non-negative values before log transform
            if (df_prep[col] < 0).any():
                print(f"   - Warning: Column '{col}' contains negative values. Skipping log transformation.")
                continue
                
            new_col_name = f"{col}_log"
            df_prep[new_col_name] = np.log1p(df_prep[col])
            transformed_cols_log.append(new_col_name)
            original_cols_to_potentially_drop.append(col) # Mark original for potential drop later
            
        else:
            print(f"   - Warning: Column '{col}' not found.")
            
    print(f"\nCreated {len(transformed_cols_log)} new log-transformed columns.")
    print("Original columns kept for now, consider dropping later if log version proves superior.")

    # Verify results - describe new log columns
    if transformed_cols_log:
        print("\nDescriptive statistics for new log-transformed columns:")
        display(df_prep[transformed_cols_log].describe())
        
    # Optional: Check skewness again after transformation
    # if transformed_cols_log:
    #    new_skewness = df_prep[transformed_cols_log].skew()
    #    print("\nSkewness AFTER log transformation:")
    #    display(new_skewness)

else:
    print("Error: df_prep DataFrame not found or skewed columns not identified.")

Applying log1p transformation to 12 highly skewed columns:
['calculated_host_listings_count_shared_rooms', 'bathrooms', 'calculated_host_listings_count_private_rooms', 'host_acceptance_rate', 'beds', 'days_since_last_review', 'number_of_reviews', 'bedrooms', 'reviews_per_month', 'accommodates', 'calculated_host_listings_count_entire_homes', 'number_of_reviews_ltm']

Created 12 new log-transformed columns.
Original columns kept for now, consider dropping later if log version proves superior.

Descriptive statistics for new log-transformed columns:


,calculated_host_listings_count_shared_rooms_log,bathrooms_log,calculated_host_listings_count_private_rooms_log,host_acceptance_rate_log,beds_log,days_since_last_review_log,number_of_reviews_log,bedrooms_log,reviews_per_month_log,accommodates_log,calculated_host_listings_count_entire_homes_log,number_of_reviews_ltm_log
count,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000,8768.000
mean,0.042,0.793,0.381,0.652,1.132,4.152,3.285,0.800,0.922,1.532,1.967,2.346
std,0.388,0.207,0.828,0.118,0.471,2.096,1.743,0.353,0.638,0.432,1.334,1.469
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.693,0.000,0.000
25%,0.000,0.693,0.000,0.678,0.693,2.708,2.079,0.693,0.336,1.099,0.693,1.099
50%,0.000,0.693,0.000,0.693,1.099,3.638,3.555,0.693,0.916,1.609,1.792,2.639
75%,0.000,0.916,0.000,0.693,1.386,5.081,4.644,1.099,1.456,1.792,2.996,3.638
max,4.159,2.773,4.078,0.693,3.497,9.210,7.562,2.565,3.742,2.833,4.625,5.805


*Observation:* Log transformation (`log1p`) was applied to 12 numeric features identified as highly skewed, including counts (`number_of_reviews*`, `calculated_host_listings_count_*`, `reviews_per_month`, `num_amenities`, `num_host_verifications`), size/capacity metrics (`accommodates`, `bathrooms`, `bedrooms`, `beds`), acceptance rate (`host_acceptance_rate`), and recency (`days_since_last_review`). New columns ending in `_log` were created. The descriptive statistics for these transformed columns show significantly compressed ranges and likely reduced skewness (indicated by closer mean and median values) compared to their original scales, making them potentially better suited for modeling. Original columns were retained for now.

### Verify Skewness Reduction
Re-calculate skewness for the potentially problematic numeric columns (using the capped/log-transformed versions where applicable) to verify that the previous steps have reduced extreme skewness.

In [32]:
# P5.3 Verify Skewness Reduction
if 'df_prep' in locals() and df_prep is not None:
    
    # Define the set of numeric features to check *after* transformations/capping
    # These are the original names OR their _log counterparts if created
    
    features_to_check_final_skew = []
    original_features_processed = [
         'accommodates', 'bathrooms', 'bedrooms', 'beds', 
         'minimum_nights', 'maximum_nights', # Use capped versions
         'number_of_reviews', 'number_of_reviews_ltm', 
         'calculated_host_listings_count_entire_homes', 
         'calculated_host_listings_count_private_rooms', 
         'calculated_host_listings_count_shared_rooms', 
         'reviews_per_month', 
         'num_amenities', 
         'num_host_verifications',
         'host_duration_days', 
         'days_since_last_review',
         'host_response_rate', # Check original or log if transformed
         'host_acceptance_rate' # Check original or log if transformed
        ]

    for col in original_features_processed:
        log_col = f"{col}_log"
        if log_col in df_prep.columns:
             features_to_check_final_skew.append(log_col) # Prefer the log version if it exists
        elif col in df_prep.columns and pd.api.types.is_numeric_dtype(df_prep[col]):
             features_to_check_final_skew.append(col) # Otherwise use the original (potentially capped)

    # Ensure no duplicates and all exist
    features_to_check_final_skew = sorted(list(set(features_to_check_final_skew)))
    features_to_check_final_skew = [f for f in features_to_check_final_skew if f in df_prep.columns]

    if features_to_check_final_skew:
        print("Calculating skewness for numeric features after capping/transformation:")
        final_skewness = df_prep[features_to_check_final_skew].skew().sort_values(key=abs, ascending=False)
        
        print("\nSkewness values:")
        display(final_skewness)
        
        # Identify columns still highly skewed (threshold might be lower now, e.g., 0.75 or 1.0)
        remaining_skew_threshold = 1.0 
        still_skewed_cols = final_skewness[abs(final_skewness) > remaining_skew_threshold]
        
        if not still_skewed_cols.empty:
             print(f"\nWarning: The following columns still have absolute skewness > {remaining_skew_threshold}:")
             display(still_skewed_cols)
             print("Further transformation or reliance on robust models might be needed.")
        else:
             print(f"\nSuccess: All checked features now have absolute skewness <= {remaining_skew_threshold}.")
             
    else:
        print("Could not identify relevant numeric columns to check final skewness.")

else:
    print("Error: df_prep DataFrame not found.")

Calculating skewness for numeric features after capping/transformation:

Skewness values:


calculated_host_listings_count_shared_rooms_log     9.982
minimum_nights                                      5.331
host_acceptance_rate_log                           -3.982
bathrooms_log                                       2.500
calculated_host_listings_count_private_rooms_log    2.277
days_since_last_review_log                          1.055
beds_log                                            0.760
maximum_nights                                      0.648
accommodates_log                                    0.556
calculated_host_listings_count_entire_homes_log     0.349
number_of_reviews_log                              -0.335
number_of_reviews_ltm_log                          -0.331
num_host_verifications                              0.292
host_duration_days                                 -0.165
reviews_per_month_log                               0.132
bedrooms_log                                        0.096
num_amenities                                       0.025
dtype: float64

calculated_host_listings_count_shared_rooms_log     9.982
minimum_nights                                      5.331
host_acceptance_rate_log                           -3.982
bathrooms_log                                       2.500
calculated_host_listings_count_private_rooms_log    2.277
days_since_last_review_log                          1.055
dtype: float64

Further transformation or reliance on robust models might be needed.


*Observation:* The skewness check after capping and log transformations reveals that while many features improved, several columns (`calculated_host_listings_count_shared_rooms_log`, `minimum_nights`, `host_acceptance_rate_log`, `bathrooms_log`, `calculated_host_listings_count_private_rooms_log`, `days_since_last_review_log`) still exhibit absolute skewness greater than 1.0. This is often due to distributions dominated by zero/low values or the nature of capping/imputation. **Decision:** Given that tree-based models (Random Forest, Gradient Boosting) are primary candidates for this type of prediction task and are generally robust to monotonic transformations and skewness, no further transformations will be applied at this stage. We will proceed with the current feature set, acknowledging the remaining skewness in these specific columns.

### Display Info and Head
Show the updated DataFrame info and head to see the newly added features and removed original columns.

In [33]:
if 'df_prep' in locals() and df_prep is not None:
    print("\nDataFrame head after Phase P4 Feature Engineering & Parsing:")
    display(df_prep.head())
    
    print("\nDataFrame info after Phase P4 Feature Engineering & Parsing:")
    df_prep.info()
else:
    print("Error: df_prep DataFrame not found.")


DataFrame head after Phase P4 Feature Engineering & Parsing:


,host_response_time,host_acceptance_rate,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,has_availability,availability_30,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,log_price,num_amenities,amenity_wifi,amenity_kitchen,amenity_air_conditioning,amenity_heating,amenity_washer,amenity_dryer,amenity_tv,amenity_parking,amenity_pool,amenity_pets_allowed,amenity_long_term_stays_allowed,num_host_verifications,host_duration_days,days_since_last_review,neighbourhood_group,has_reviews,calculated_host_listings_count_shared_rooms_log,bathrooms_log,calculated_host_listings_count_private_rooms_log,host_acceptance_rate_log,beds_log,days_since_last_review_log,number_of_reviews_log,bedrooms_log,reviews_per_month_log,accommodates_log,calculated_host_listings_count_entire_homes_log,number_of_reviews_ltm_log
0,within_hour,1.000,1,Praha 1,50.082,14.416,Entire rental unit,Entire_home/apt,4,1.000,1.000,2.000,1,365,1,0,0,31,1,4.900,4.930,4.860,1,69,0,0,0.180,7.979,30,1,1,1,1,1,1,1,0,0,0,1,2,5933,274.000,Old_Town_Center,1,0.000,0.693,0.000,0.693,1.099,5.617,3.466,0.693,0.166,1.609,4.248,0.693
2,within_hour,0.980,1,Praha 1,50.087,14.432,Entire rental unit,Entire_home/apt,4,1.500,1.000,2.000,3,700,1,3,173,411,53,4.940,4.930,4.900,0,3,0,0,3.430,7.367,58,1,1,1,1,1,1,1,1,0,0,1,2,5438,9.000,Old_Town_Center,1,0.000,0.916,0.000,0.683,1.099,2.303,6.021,0.693,1.488,1.609,1.386,3.989
3,within_hour,0.800,1,Praha 3,50.087,14.445,Private room in rental unit,Private_room,2,1.000,1.000,2.000,3,60,1,5,5,414,52,4.760,4.630,4.830,0,3,3,0,2.790,6.758,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,14.000,Near_Center_East,1,0.000,0.693,1.386,0.588,1.099,2.708,6.028,0.693,1.332,1.099,1.386,3.970
4,within_hour,0.800,1,Praha 3,50.085,14.445,Private room in rental unit,Private_room,2,1.000,1.000,3.000,3,60,1,3,3,389,47,4.690,4.590,4.730,0,3,3,0,2.670,6.446,31,1,0,0,1,0,1,1,1,0,0,0,2,4510,15.000,Near_Center_East,1,0.000,0.693,1.386,0.588,1.386,2.773,5.966,0.693,1.300,1.099,1.386,3.871
5,within_hour,0.800,1,Praha 3,50.085,14.446,Private room in rental unit,Private_room,2,1.000,1.000,1.000,3,60,1,6,6,381,52,4.780,4.680,4.810,0,3,3,0,2.580,6.650,29,1,0,0,1,0,1,1,1,0,0,0,2,4510,21.000,Near_Center_East,1,0.000,0.693,1.386,0.588,0.693,3.091,5.945,0.693,1.275,1.099,1.386,3.970



DataFrame info after Phase P4 Feature Engineering & Parsing:
<class 'pandas.core.frame.DataFrame'>
Index: 8768 entries, 0 to 10107
Data columns (total 57 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   host_response_time                                8768 non-null   object 
 1   host_acceptance_rate                              8768 non-null   float64
 2   host_identity_verified                            8768 non-null   int64  
 3   neighbourhood_cleansed                            8768 non-null   object 
 4   latitude                                          8768 non-null   float64
 5   longitude                                         8768 non-null   float64
 6   property_type                                     8768 non-null   object 
 7   room_type                                         8768 non-null   object 
 8   accommodates                            

In [34]:
# Save dataset state AFTER applying outlier capping and log transformations
if 'df_prep' in locals() and df_prep is not None:
    save_path_after = '../data/interim/dataset_after_outliers.csv'
    print(f"Saving DataFrame state AFTER outlier/skew handling to: {save_path_after}")
    try:
        # Saving the current state of df_prep
        df_prep.to_csv(save_path_after, index=False)
        print(f"DataFrame state saved successfully. Shape: {df_prep.shape}")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
else:
    print("Error: df_prep DataFrame not found. Cannot save state after outlier handling.")

Saving DataFrame state AFTER outlier/skew handling to: ../data/interim/dataset_after_outliers.csv
DataFrame state saved successfully. Shape: (8768, 57)


## 8. Encoding Categorical Features

This phase addresses potential outliers and high skewness identified in the predictor variables. Handling these can improve model stability and performance. We will:
*   Programmatically identify numeric features exhibiting high skewness or extreme maximum values.
*   Cap extreme values in stay duration columns (`minimum_nights`, `maximum_nights`).
*   Apply log transformations to other highly skewed numeric features identified.